In [1]:
import ollama
from typing import TypedDict,List
from langgraph.graph import StateGraph
from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS



Imports all tools used in the pipeline

In [4]:
PDF_PATH='Ebook-Agentic-AI.PDF'

loader=PyPDFLoader(PDF_PATH)
docs=loader.load()

print('PDF loaded:',len(docs),'pages')

PDF loaded: 60 pages


Reads the pdf file and extracts text content.

In [5]:
splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
chunks=splitter.split_documents(docs)

print('chunks created:',len(chunks))

chunks created: 195


Breaks long text into small meaningful chunks

In [6]:
embedding=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

Converts text chunks into numerical vectors using a local embedding model.

In [7]:
db=FAISS.from_documents(chunks,embedding)

print('stored in FAISS')

stored in FAISS


Stores all chunk vectors in FAISS, a local vector search engine.

In [8]:
class RAGState(TypedDict):
    query:str
    retrieved_docs:List
    answer:str

In [13]:
def retrieve(state:RAGState):
    query=state['query']
    docs_with_scores=db.similarity_search_with_score(query,k=3)
    return {'retrieved_docs': docs_with_scores}

In [16]:
def generate_answer(state:RAGState):
    query=state['query']
    docs_with_scores=state['retrieved_docs']

    context='\n\n'.join([doc.page_content for doc,score in docs_with_scores])
    prompt=f'''
Answer only using the content below.If answeris not in the text,say:
Not found in document.

Context:
{context}

Question:
{query}


'''
    response=ollama.chat(model='llama3',messages=[{'role':'user','content':prompt}])
    return {'answer':response['message']['content'],
       'retrieved_docs':docs_with_scores}


In [17]:
graph = StateGraph(RAGState)
graph.add_node('retrieve',retrieve)
graph.add_node('generate',generate_answer)
graph.set_entry_point('retrieve')
graph.add_edge('retrieve','generate')
rag_app=graph.compile()
print('Langgraph RAG ready')


Langgraph RAG ready


In [19]:
query=input('Ask a question:')
result=rag_app.invoke({'query':query})

print('Answer:')
print(result['answer'])

print('Retrieved chunks +scores:')
for i, (doc,score) in enumerate(result['retrieved_docs'], 1):
    print(f'Chunk {i} | Score:{score}')
    print(doc.page_content[:500])

Ask a question:what is agentis ai?
Answer:
According to the provided document, "What is Agentic AI?" is one of the topics covered in this section. Here's the relevant text:

"In this section, we will define what Agentic AI is and, more importantly, what it’s not, as it’s often misunderstood. While Agentic AI promises a shift from reactive to proactive problem-solving, doubts remain. Is it truly real? Are all the claims about it accurate?"

Not found in document: No additional information about "What is Agentic AI?" beyond this introductory text.
Retrieved chunks +scores:
Chunk 1 | Score:0.540095865726471
Agentic AI
An Executive's Guide to In-depth
Understanding of Agentic AI
Chunk 2 | Score:0.5558705925941467
� What value does it bring? 
                 � How are businesses using it in the real world? 
INTRODUCTION 
TO AGENTIC AI
01
Chunk 3 | Score:0.5950372815132141
In this section, we will define what Agentic AI is and, more importantly, 
what it’s not, as it’s often misunderstood. 